In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}


filename = "기업공채정보.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)


title = "기업명 내용 키워드 경력 학력 마감일 웹사이트".split(" ")
writer.writerow(title)


for page in range(1, 21):
    url = "https://www.jobkorea.co.kr/Starter/?JoinPossible_Stat=0&schOrderBy=0&LinkGubun=0&LinkNo=0&schType=0&schGid=0&Page="
    res = requests.get(url + str(page), headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
        
    enterprises = soup.find("div", attrs={"class":"filterListArea"}).find_all("li") # 각 기업들이 속한 행 전체 불러오기

    for enterprise in enterprises:
        try: 
            name = enterprise.find("div", attrs={"class":"coTit"}).find("a", attrs={"class":"coLink"}).get_text().strip() # 기업명
        
            content = enterprise.find("div", attrs={"class":"tit"}).a.get_text().strip() # 내용
            
            keyword = enterprise.find("div", attrs={"class":"sTit"}).get_text().strip().replace("\n", "·").split("·") # 키워드, \n, · 문자열 자르기
            keyword = str(keyword)[1:-1].replace("'","") # 문자열 형식으로 변환 후 [, ], ' 제거
           
            career = enterprise.find("div", attrs={"class":"sDesc"}).strong.get_text().strip() # 경력
        
            education = enterprise.find("div", attrs={"class":"sDesc"}).span.get_text().strip() # 학력
        
            if enterprise.find("div", attrs={"class":"side"}).find("span", attrs={"class":"day"}).get_text().startswith("~"): # 마감일, '~2021...' 텍스트만 추출
                duedate = enterprise.find("div", attrs={"class":"side"}).find("span", attrs={"class":"day"}).get_text()[:-3].strip() # GUI 문제 때문에 요일 삭제
            elif enterprise.find("div", attrs={"class":"side"}).find("span", attrs={"class":"day schedule"}).get_text().startswith("진행예정"): # 마감일이 진행예정일 경우
                duedate = enterprise.find("div", attrs={"class":"side"}).find("span", attrs={"class":"day"}).get_text().strip()    
                
            if enterprise.find("a", attrs={"class":"coLink"})["href"].startswith("https://"): # 웹사이트 링크, "https://"로 시작하는 경우
                link = enterprise.find("a", attrs={"class":"coLink"})["href"]
            else:
                link = "https://www.jobkorea.co.kr/" + enterprise.find("a", attrs={"class":"coLink"})["href"] # "https://"로 시작하지 않는 경우
        except:
            pass
        
        totallist = [name, content, keyword, career, education, duedate, link]
        writer.writerow(totallist)
        
        
df_csv = pd.read_csv("기업공채정보.csv", header = 0, index_col = False)
display(df_csv)

,기업명,내용,키워드,경력,학력,마감일,웹사이트
0,한영회계법인,[EY한영] 전략·재무자문본부 TCF부문 인턴 모집,"회계, 세무, CPA, 재무, 세무, IR",인턴·신입,대졸↑,~2021.06.18,https://www.jobkorea.co.kr/Company/16151846
1,금양인터내셔날,[금양인터내셔날] 마케팅기획팀 대리/과장급 정규직 사원 모집,"마케팅, 광고, 분석, 홍보, PR, 기획, 전략, 경영",경력,대졸↑,~2021.06.20,https://www.jobkorea.co.kr//Recruit/Co_Read/C/...
2,㈜오토닉스,2021년도 상반기 상시채용,"소프트웨어, 하드웨어, HTML, 퍼블리싱, UI개발, 기획, 전략, 경영",신입·경력,초대졸↑,~2021.07.30,https://www.jobkorea.co.kr/Company/1735925
3,홈플러스㈜,[경력직] 미들웨어 전문가 채용,"컨텐츠, 사이트운영",경력,대졸↑,~2021.06.21,https://www.jobkorea.co.kr/Company/1932979
4,홈플러스㈜,보안관제 운영관리 경력직 채용,"네트워크, 서버, 보안",경력,대졸↑,~2021.06.21,https://www.jobkorea.co.kr/Company/1932979
...,...,...,...,...,...,...,...
995,㈜소노인터내셔널,[쏠비치 양양] 2021년 상반기 인재 채용,"여행, 항공, 숙박, 판매, 서빙, 매장관리, 요리, 영양, 제과제빵, 바리스타",신입·경력,초대졸↑,~2021.09.01,https://www.jobkorea.co.kr/Company/1938839
996,㈜소노인터내셔널,[쏠비치 양양] 2021년 상반기 인재 채용,"여행, 항공, 숙박, 판매, 서빙, 매장관리, 요리, 영양, 제과제빵, 바리스타",신입·경력,초대졸↑,~2021.09.01,https://www.jobkorea.co.kr/Company/1938839
997,㈜소노인터내셔널,[쏠비치 양양] 2021년 상반기 인재 채용,"여행, 항공, 숙박, 판매, 서빙, 매장관리, 요리, 영양, 제과제빵, 바리스타",신입·경력,초대졸↑,~2021.09.01,https://www.jobkorea.co.kr/Company/1938839
998,㈜소노인터내셔널,[쏠비치 양양] 2021년 상반기 인재 채용,"여행, 항공, 숙박, 판매, 서빙, 매장관리, 요리, 영양, 제과제빵, 바리스타",신입·경력,초대졸↑,~2021.09.01,https://www.jobkorea.co.kr/Company/1938839


In [2]:
from tkinter import *
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
import tkinter
import tkinter.ttk as ttk
import pandas as pd
import sqlite3


def donothing():
    pass

def close():
    window.quit()
    window.destroy()

# DB 생성 TABLE 생성 
def create_table():
    try:
        con = sqlite3.connect("project_최종1.db")
        cursor = con.cursor()
        sql = """CREATE TABLE 기업공채정보(기업명 TEXT, 내용 TEXT, 키워드  TEXT, 경력 TEXT, 학력 TEXT, 마감일 TEXT, 웹사이트 TEXT)"""
        cursor.execute(sql)
    except:
        pass
    finally:
        con.commit()
        con.close()

create_table()


# GUI상에서 open -> csv파일 선택하여 DB에 row별로 넣는 함수

def openFile():
    global fname
    fname = askopenfilename(filetypes=(("csv","*.csv"), ("All Files","*.*") ))
    df= pd.read_csv("기업공채정보.csv")
    try:
        con = sqlite3.connect('project_최종1.db')
        cursor = con.cursor()
        sql ="INSERT INTO 기업공채정보 VALUES(?,?,?,?,?,?,?)"
        for row in df.values:
            cursor.execute(sql, tuple(row))
    except:
        pass
    
    finally:
        con.commit()
        con.close()
        messagebox.showinfo("DB 연결", fname)


# 신입 or 경력무관     
def career_no():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 경력 LIKE "%신입%"OR 경력 = "경력무관"')
        a = cur.fetchall()
        return a    
    except:
        pass
    finally:
        con.close()

# 경력 or 경력무관
def career_yes():
    try:    
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 경력 LIKE "%경력%"')
        b = cur.fetchall()
        return b    
    except:
        pass
    finally:
        con.close()


# 키워드 if entry1 -> 입력받으면 관련 정보만 출력 else: 전체 출력 (search에서 intersection떄문에) 
def keyword():
    if entry1.get():
        x = entry1.get()
        y = '%'+str(x)+'%'
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 키워드 LIKE "%s"'%y)
        c = cur.fetchall()
        con.close()
        return set(c)
    else:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보')
        c = cur.fetchall()
        con.close()
        return set(c)    

# 학력
def education_high():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 학력 = "고졸↑"')
        d = cur.fetchall()
        return d
    except:
        pass
    finally:
        con.close()

def education_col():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 학력 = "초대졸↑"')
        e = cur.fetchall()
        return e
    except:
        pass
    finally:
        con.close()
        


def education_uni():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 학력 = "대졸↑" ')
        f = cur.fetchall()
        return f
    except:
        pass
    finally:
        con.close()

def education_mas():
    try:  
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 학력 = "석사↑" OR 학력 = "박사↑"')
        g = cur.fetchall()
        return g
    except:
        pass
    finally:
        con.close()



def education_no():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 학력 = "학력무관" ')
        h = cur.fetchall()
        return h
    except:
        pass
    finally:
        con.close()



# 마감시한 -> entry 입력값 yyyy.mm.dd(요일) 로 입력받음 
#-> #datetime을 통해 요일 계산되게 짜기보단 그냥 csv파일을 뽑을때 요일부분을 없애줬으면 좋겠음.( 현섭씨한테 부탁드려볼게요.)
# keyword랑 동일하게 진행하여 짯음
def duedate():
    if entry2.get():
        x =entry2.get()
        y = '~'+ str(x)
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 마감일 FROM 기업공채정보 ORDER BY 마감일')
        a = cur.fetchone()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 WHERE 마감일 BETWEEN "%s"  AND  "%s"'%(a,y))
        i = cur.fetchall()
        return set(i)
    else:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보 ')
        i = cur.fetchall()
        con.close()
        return set(i)

def all_info():
    try:
        con = sqlite3.connect("project_최종1.db")
        cur = con.cursor()
        cur.execute('SELECT DISTINCT 기업명,내용,웹사이트 FROM 기업공채정보')
        j= cur.fetchall()
        return set(j)
    except:
        pass
    finally:
        con.close()
    
# career_no_var라는 IntVar를 만들어주고 checkbutton의 입력값 받아오기 위해서 짯음 
#-> 리스트안의 튜플형식을 세트형식으로 바꿔서 intersection, union진행 -> 리스트형태
# https://stackoverflow.com/questions/4236910/getting-checkbutton-state 관련 참고하시면 됩니다.  
#( root = Tk()를 안하고 진행하면 에러떠서 생성했는데 windw를 root로 바꿔도 되는지 찾아보기 -> 출력 쪼꼬미창 제거) 
#-> 주석 처리해도 오류가 발생하지 않아서 주석 처리했습니다._양봉근 2021.06.09 17:22 -> 확인완료_석호준

def search():
    if career_no_var.get():
        set_career_no = set(career_no())
    else:
        set_career_no = set()

    if career_yes_var.get():
        set_career_yes = set(career_yes())
    else:
        set_career_yes = set()

    if education_high_var.get():
        set_education_high = set(education_high())
    else:
        set_education_high = set()

    if education_col_var.get():
        set_education_col = set(education_col())
    else:
        set_education_col = set()

    if education_uni_var.get():
        set_education_uni = set(education_uni())
    else:
        set_education_uni = set()
        
    if education_mas_var.get():
        set_education_mas = set(education_mas())
    else:
        set_education_mas = set()

    if education_no_var.get():
        set_education_no = set(education_no())
    else:
        set_education_no = set()

    set_keyword = keyword()
    set_duedate = duedate()
    set_all_info = all_info()
    
    if set_career_no == set() and set_career_yes == set() and set_education_high!= set() and set_education_col!=set() and set_education_uni!= set() and set_education_mas!= set() and set_education_no!= set():
        part1 = set_all_info
        result = list((part1)&(set_education_high | set_education_col | set_education_uni | set_education_mas | set_education_no)&(set_keyword)&(set_duedate))
    elif set_career_no != set() and set_career_yes != set() and set_education_high== set() and set_education_col==set() and set_education_uni== set() and set_education_mas== set() and set_education_no== set():
        part2 = set_all_info
        result = list((set_career_no | set_career_yes)& (part2)&(set_keyword)&(set_duedate))
    elif set_career_no== set() and set_career_yes== set() and set_education_high== set() and set_education_col== set() and set_education_uni== set() and set_education_mas== set() and set_education_no== set():
        part1 = set_all_info
        part2 = set_all_info
        result = list((part1)& (part2)&(set_keyword)&(set_duedate))
    elif set_career_no == set() and set_career_yes == set():
        part1 = set_all_info
        result = list((part1)&(set_education_high | set_education_col | set_education_uni | set_education_mas | set_education_no)&(set_keyword)&(set_duedate))    
    elif set_education_high== set() and set_education_col==set() and set_education_uni== set() and set_education_mas== set() and set_education_no== set():
        part2 = set_all_info
        result =  list((set_career_no | set_career_yes)& (part2)&(set_keyword)&(set_duedate))
    else:
        result =list((set_career_no | set_career_yes)&(set_education_high | set_education_col | set_education_uni | set_education_mas | set_education_no)&(set_keyword)&(set_duedate))
        
    for i in range(len(result)):
        tv.insert('','end',text=i,values=result[i])




# retry 고쳐드렸습니다. https://stackoverflow.com/questions/22812134/how-to-clear-an-entire-treeview-with-tkinter 참고하세요. 2021.06.09 20:52 석호준      
def retry():
    for i in tv.get_children():
        tv.delete(i)


########################################################

window = Tk()
window.title("Job Search System Project")
window.geometry("800x700")
window.resizable(0,0)


menubar = Menu(window)

filemenu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="File", menu=filemenu)
filemenu.add_command(label="New", command=donothing)
filemenu.add_command(label="Open", command=openFile)
filemenu.add_command(label="Save", command=donothing)
filemenu.add_command(label="Save as...", command=donothing)
filemenu.add_separator()
filemenu.add_command(label="Exit", command=close)

filemenu1 = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Etc", menu=filemenu1)
filemenu1.add_command(label="New", command=donothing)

window.config(menu=menubar)

# Label 생성하기
Label(window, text="직업 검색 도우미",font=("돋움체",20)).pack(padx=5, pady=15)

labelCareer = Label(window, text="경력 : ")
labelCareer.place(x = 100, y = 100)
labelAcademic = Label(window, text="학력 : ")
labelAcademic.place(x = 100, y = 130)
labelDuty = Label(window, text="키워드 (ex) 마케팅) : ")
labelDuty.place(x = 100, y = 160)
labelDeadline = Label(window, text="마감시한 (ex) 'yyyy.mm.dd') : ")
labelDeadline.place(x = 100, y = 190)
labelResult = Label(window, text="<검색 결과>")
labelResult.place(x=100, y=270)

####################################################
#root = Tk() -> 주석 처리해도 오류가 발생하지 않아서 주석 처리했습니다._양봉근 2021.06.09 17:22 
career_no_var = tkinter.IntVar() 
career_yes_var = tkinter.IntVar()
education_high_var = tkinter.IntVar()
education_col_var = tkinter.IntVar()
education_uni_var = tkinter.IntVar()
education_mas_var = tkinter.IntVar()
education_no_var = tkinter.IntVar()

#commnad(X) variable(o)
checkbutton_widget1 = Checkbutton(window, text = "신입",variable= career_no_var, activebackground="yellow")
checkbutton_widget2 = Checkbutton(window, text = "경력",variable=  career_yes_var, activebackground="yellow")
checkbutton_widget3 = Checkbutton(window, text = "고등학교졸업",variable=  education_high_var, activebackground="yellow")
checkbutton_widget4 = Checkbutton(window, text = "대학졸업(2,3년)",variable=  education_col_var, activebackground="yellow")
checkbutton_widget5 = Checkbutton(window, text = "대학교졸업(4년)",variable= education_uni_var, activebackground="yellow")
checkbutton_widget6 = Checkbutton(window, text = "석/박사 졸업",variable=  education_mas_var, activebackground="yellow")
checkbutton_widget7 = Checkbutton(window, text = "학력무관",variable= education_no_var, activebackground="yellow")
####################################################  GUI 수정하실꺼면 아래에서부터 수정하세요 ##사이 건들X 



checkbutton_widget1.place(x = 140, y = 100)
checkbutton_widget2.place(x = 190, y = 100)
checkbutton_widget3.place(x = 140, y = 130)
checkbutton_widget4.place(x = 240, y = 130)
checkbutton_widget5.place(x = 350, y = 130)
checkbutton_widget6.place(x = 465, y = 130)
checkbutton_widget7.place(x = 560, y = 130)

# entry, text 박스 생성
entry1= Entry(window)
entry2 =Entry(window)
Output = Text(window, width=20, wrap=CHAR)

entry1.place(x = 220, y = 160)
entry2.place(x = 270, y = 190)
Output.place(x= 100, y= 300)

#Treeview 만들기
tv = ttk.Treeview(Output)
tv.pack()

tv_scb = ttk.Scrollbar(tv, orient="vertical", command=tv.yview)
tv_scb.place(relx=0.978, rely=0.100, relheight=0.893, relwidth=0.020)

tv.configure(yscrollcommand=tv_scb.set)

tv['columns']= ("Name", "Contents", "URL")

tv.column("#0", anchor=CENTER, width=80, minwidth=50)
tv.column("Name", anchor=CENTER, width=100, minwidth=50)
tv.column("Contents", anchor=CENTER, width=150, minwidth=50)
tv.column("URL", anchor=CENTER, width=230, minwidth=100)


tv.heading("#0", text="Index", anchor=CENTER)
tv.heading("Name",text="기업명", anchor=CENTER)
tv.heading("Contents",text="내용",anchor=CENTER)
tv.heading("URL",text="웹사이트", anchor=CENTER)


# 검색하기 버튼 생성
btn1 = Button(window, text = "선택 조건으로 검색",relief=RAISED, command = search)
btn1.place(x = 480, y = 190)
btn1['bg']='skyblue'
btn2 = Button(window, text= '다시 검색',relief=RAISED, command=retry)
btn2.place(x = 600, y = 190)


window.mainloop()
